https://medium.com/@franky07724_57962/using-keras-pre-trained-models-for-feature-extraction-in-image-clustering-a142c6cdf5b1

In [1]:
#Importing the ResNet50 model
from keras.applications.resnet50 import ResNet50, preprocess_input

#Loading the ResNet50 model with pre-trained ImageNet weights
# include_top = False loads until the penultimate layer
model = ResNet50(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

Using TensorFlow backend.


94658560/94653016 [==============================] - 394s 4us/step


In [15]:
#scaler = transforms.Scale((224, 224))
#normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                 std=[0.229, 0.224, 0.225])
#to_tensor = transforms.ToTensor()
import numpy as np
from PIL import Image
pic_one_path = '../Data/hymenoptera_data/train/ants/swiss-army-ant.jpg'

image = Image.open(pic_one_path)

print("Creating numpy representation of image {}".format(pic_one_path))
resize = image.resize((200,200), Image.NEAREST) 
resize.load()
data = np.asarray(resize, dtype="uint8" )
print(data.shape)

Creating numpy representation of image ../Data/hymenoptera_data/train/ants/swiss-army-ant.jpg
(200, 200, 3)


In [23]:

#Reshaping the training data
#X_train_new = np.array([imresize(X_train[i], (200, 200, 3)) for i in range(0, len(X_train))]).astype('float32')

X_train_new = np.array([data]).astype('float32')

#Preprocessing the data, so that it can be fed to the pre-trained ResNet50 model. 
resnet_train_input = preprocess_input(X_train_new)

#Creating bottleneck features for the training data
train_features = model.predict(resnet_train_input)

vector = train_features.reshape(1,2048)

#Saving the bottleneck features
np.savez('resnet_features_train', features=train_features)
vector.shape

(1, 2048)

In [22]:
print("type(train_features) = {} and train_features.shape = {}".format(type(train_features), train_features.shape))

type(train_features) = <class 'numpy.ndarray'> and train_features.shape = (1, 1, 1, 2048)


In [60]:
#Importing the ResNet50 model
from keras.applications.resnet50 import ResNet50, preprocess_input

#Loading the ResNet50 model with pre-trained ImageNet weights
# include_top = False loads until the penultimate layer
model = ResNet50(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

In [65]:
def get_vector(image_path):
    # 1. Load the image with Pillow library
    image = Image.open(image_path)
    
    # 2. Resize image to the shape acceptable as input to the ResNet50 model
    resized_image = image.resize((200, 200), Image.NEAREST)
    resized_image.load()
    
    # 3. Preprocess the resized image so that in can be fed as an input to ResNet50
    resized_image = np.asarray(resized_image, dtype="uint8" )
    X = np.array([resized_image]).astype('float32')
    resnet_input = preprocess_input(X)
    
    # 4. Forward pass through model
    last_layer_activations = model.predict(resnet_input)
    
    # 5. Reshape the activations to 1 dimensional vector
    vector = last_layer_activations.reshape(1, 2048)
    
    return vector

In [66]:
pic_one_path = '../Data/hymenoptera_data/train/ants/swiss-army-ant.jpg'
pic_two_path = '../Data/hymenoptera_data/train/ants/Ant_1.jpg'
pic_three_path = '../Data/hymenoptera_data/train/bees/98391118_bdb1e80cce.jpg'

In [67]:
v1 = get_vector(pic_one_path)
v2 = get_vector(pic_two_path)
v3 = get_vector(pic_three_path)

In [68]:
type(v1)
v1.shape

(1, 2048)

In [71]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html
from sklearn.metrics.pairwise import cosine_similarity

v_cat = get_vector('/home/raghav/Downloads/cat.jpeg')
v_dog = get_vector('/home/raghav/Downloads/dog.jpeg')

print("similarity between cat and dog = {}".format(np.asscalar(cosine_similarity(v_cat, v_dog))))
print("similarity between two ants = {}".format(np.asscalar(cosine_similarity(v1, v2))))
print("similarity between ant and bee = {}".format(np.asscalar(cosine_similarity(v1, v3))))

similarity between cat and dog = 0.31559327244758606
similarity between two ants = 0.2525273561477661
similarity between ant and bee = 0.45515644550323486


In [45]:
v1.tolist()

[[2.421546220779419,
  0.678417444229126,
  0.1392480731010437,
  0.4559420645236969,
  0.06805622577667236,
  0.4140651226043701,
  0.1600290983915329,
  0.02957269176840782,
  0.21410363912582397,
  0.39937081933021545,
  0.3211480677127838,
  0.07129371911287308,
  0.1933647096157074,
  0.21193484961986542,
  0.2823941111564636,
  0.2625042200088501,
  0.006873502861708403,
  0.07700689136981964,
  0.8882542252540588,
  0.6946176290512085,
  1.2950763702392578,
  0.1548190414905548,
  0.7944557666778564,
  0.0727607011795044,
  0.008126242086291313,
  0.46341443061828613,
  0.21600095927715302,
  0.005891984794288874,
  1.9453918933868408,
  0.0,
  0.04591517522931099,
  0.03273513540625572,
  0.9056083559989929,
  0.056763872504234314,
  0.014336006715893745,
  0.03935055434703827,
  0.1396113485097885,
  0.25804346799850464,
  0.7243772745132446,
  1.8788094520568848,
  0.0,
  0.0,
  0.0,
  0.053660374134778976,
  0.6925836801528931,
  0.4482001066207886,
  0.28797048330307007,
  

In [46]:
v2.tolist()

[[2.421546220779419,
  0.678417444229126,
  0.1392480731010437,
  0.4559420645236969,
  0.06805622577667236,
  0.4140651226043701,
  0.1600290983915329,
  0.02957269176840782,
  0.21410363912582397,
  0.39937081933021545,
  0.3211480677127838,
  0.07129371911287308,
  0.1933647096157074,
  0.21193484961986542,
  0.2823941111564636,
  0.2625042200088501,
  0.006873502861708403,
  0.07700689136981964,
  0.8882542252540588,
  0.6946176290512085,
  1.2950763702392578,
  0.1548190414905548,
  0.7944557666778564,
  0.0727607011795044,
  0.008126242086291313,
  0.46341443061828613,
  0.21600095927715302,
  0.005891984794288874,
  1.9453918933868408,
  0.0,
  0.04591517522931099,
  0.03273513540625572,
  0.9056083559989929,
  0.056763872504234314,
  0.014336006715893745,
  0.03935055434703827,
  0.1396113485097885,
  0.25804346799850464,
  0.7243772745132446,
  1.8788094520568848,
  0.0,
  0.0,
  0.0,
  0.053660374134778976,
  0.6925836801528931,
  0.4482001066207886,
  0.28797048330307007,
  

In [47]:
v3.tolist()

[[2.421546220779419,
  0.678417444229126,
  0.1392480731010437,
  0.4559420645236969,
  0.06805622577667236,
  0.4140651226043701,
  0.1600290983915329,
  0.02957269176840782,
  0.21410363912582397,
  0.39937081933021545,
  0.3211480677127838,
  0.07129371911287308,
  0.1933647096157074,
  0.21193484961986542,
  0.2823941111564636,
  0.2625042200088501,
  0.006873502861708403,
  0.07700689136981964,
  0.8882542252540588,
  0.6946176290512085,
  1.2950763702392578,
  0.1548190414905548,
  0.7944557666778564,
  0.0727607011795044,
  0.008126242086291313,
  0.46341443061828613,
  0.21600095927715302,
  0.005891984794288874,
  1.9453918933868408,
  0.0,
  0.04591517522931099,
  0.03273513540625572,
  0.9056083559989929,
  0.056763872504234314,
  0.014336006715893745,
  0.03935055434703827,
  0.1396113485097885,
  0.25804346799850464,
  0.7243772745132446,
  1.8788094520568848,
  0.0,
  0.0,
  0.0,
  0.053660374134778976,
  0.6925836801528931,
  0.4482001066207886,
  0.28797048330307007,
  